In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [6]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import cv2

from data.scannet.utils_scannet import ScanNetDataset
from descriptors.classical_descriptors import (SIFT_detector, ORB_detector,
                                               SIFT_descriptor, ORB_descriptor, apply_modular_extractor_to_image)
from matching.classic.nn_matching import NNMatcherPair
from utils.metrics import calculate_metrics_simple, mAA_max, mAA_sep
from utils.matching import plot_matches

### ScanNet test

In [4]:
test_data = ScanNetDataset(
    root_dir='/home/project/ScanNet/scannet_test_1500/',
    npz_path='/home/project/ScanNet/test.npz',
    intrinsics_path='/home/project/ScanNet/intrinsics_test.npz',
    mode='test',
    mode_rgb_read='gray'
)

### SIFT

In [5]:
config = dict(
    name="SIFT-SIFT-Lowe-MMN-MAGSAC",
    normtype="l2",
    crosscheck=True,
    ratio=0.9, 
    metrics=dict(
        method=cv2.USAC_MAGSAC,
        prob=0.99999,
        threshold=3., 
        maxiters=100000
        )
)


nn_matcher = NNMatcherPair(normtype=config['normtype'], ratio=config['ratio'])
metrics = {}

for i in tqdm(range(len(test_data))):
    data = test_data[i]
    
    img_0 = cv2.normalize(data['image0'].numpy().squeeze(), None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    img_1 = cv2.normalize(data['image1'].numpy().squeeze(), None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    
    kp_0, c_0, desc_0 = apply_modular_extractor_to_image(
        img_0,
        detector=SIFT_detector,
        descriptor=SIFT_descriptor, 
        detector_params={'nfeatures':2000},
        descriptor_params={'nfeatures':2000}
    )
    
    kp_1, c_1, desc_1 = apply_modular_extractor_to_image(
        img_1,
        detector=SIFT_detector,
        descriptor=SIFT_descriptor, 
        detector_params={'nfeatures':2000},
        descriptor_params={'nfeatures':2000}
    )
    
    match_0, match_1, c_0, c_1 = nn_matcher.match(kp_0, kp_1, desc_0, desc_1, c_0, c_1) #Nx2, Nx2, Nx1, Nx1
   
    
    metrics[i] = calculate_metrics_simple(
                 match_0.astype('int').T, match_1.astype('int').T,
                 data['T_0'].numpy(), data['T_1'].numpy(),
                 data['K0'], data['K1'],
                 **config['metrics']
    )

metrics = pd.DataFrame.from_dict(metrics, orient='index')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [06:30<00:00,  3.85it/s]


In [8]:
print(f'{"mAA 5 rotation":<20} {mAA_sep(metrics["R_distance"].values, thresh=5, N=10000):.4f}')
print(f'{"mAA 10 rotation":<20} {mAA_sep(metrics["R_distance"].values, thresh=10, N=10000):.4f}')
print(f'{"mAA 20 rotation":<20} {mAA_sep(metrics["R_distance"].values, thresh=20, N=10000):.4f}')

mAA 5 rotation       0.2460
mAA 10 rotation      0.3695
mAA 20 rotation      0.4795


In [7]:
print(f'{"mAA 5 rotation":<20} {mAA_max(metrics["R_distance"].values, metrics["t_distance"].values, thresh=5, N=10000):.4f}')
print(f'{"mAA 10 rotation":<20} {mAA_max(metrics["R_distance"].values, metrics["t_distance"].values, thresh=10, N=10000):.4f}')
print(f'{"mAA 20 rotation":<20} {mAA_max(metrics["R_distance"].values, metrics["t_distance"].values, thresh=20, N=10000):.4f}')

mAA 5 rotation       0.0589
mAA 10 rotation      0.1412
mAA 20 rotation      0.2489


### ORB

In [17]:
config = dict(
    name="ORB-ORB-Lowe-MMN-MAGSAC",
    normtype="hamming",
    crosscheck=True,
    ratio=0.9, 
    metrics=dict(
        method=cv2.USAC_MAGSAC,
        prob=0.99999,
        threshold=3., 
        maxiters=100000
        )
)


nn_matcher = NNMatcherPair(normtype=config['normtype'], ratio=config['ratio'])
metrics = {}

for i in tqdm(range(len(test_data))):
    data = test_data[i]
    
    img_0 = cv2.normalize(data['image0'].numpy().squeeze(), None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    img_1 = cv2.normalize(data['image1'].numpy().squeeze(), None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    
    kp_0, c_0, desc_0 = apply_modular_extractor_to_image(
        img_0,
        detector=ORB_detector,
        descriptor=ORB_descriptor, 
        detector_params={'nfeatures':2000},
        descriptor_params={'nfeatures':2000}
    )
    
    kp_1, c_1, desc_1 = apply_modular_extractor_to_image(
        img_1,
        detector=ORB_detector,
        descriptor=ORB_descriptor, 
        detector_params={'nfeatures':2000},
        descriptor_params={'nfeatures':2000}
    )
    
    match_0, match_1, c_0, c_1 = nn_matcher.match(kp_0, kp_1, desc_0, desc_1, c_0, c_1) #Nx2, Nx2, Nx1, Nx1
   
    
    metrics[i] = calculate_metrics_simple(
                 match_0.astype('int').T, match_1.astype('int').T,
                 data['T_0'].numpy(), data['T_1'].numpy(),
                 data['K0'], data['K1'],
                 **config['metrics']
    )

metrics = pd.DataFrame.from_dict(metrics, orient='index')

print(f'{"mAA 5 rotation":<20} {mAA_max(metrics["R_distance"].values, metrics["t_distance"].values, thresh=5, N=10000):.4f}')
print(f'{"mAA 10 rotation":<20} {mAA_max(metrics["R_distance"].values, metrics["t_distance"].values, thresh=10, N=10000):.4f}')
print(f'{"mAA 20 rotation":<20} {mAA_max(metrics["R_distance"].values, metrics["t_distance"].values, thresh=20, N=10000):.4f}')

100%|███████████████████████████████████████████████████████████████████████████████| 1500/1500 [09:33<00:00,  2.62it/s]

mAA 5 rotation       0.0390
mAA 10 rotation      0.0995
mAA 20 rotation      0.1889
